# Trabalho 1 - Computação Gráfica

* Fábio Verardino de Oliveira - 12674547
* Rafael Sartori Vantin - 12543353

## Bibliotecas

In [1]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import glm
import math
from PIL import Image
from models import models_repo
from operator import itemgetter

## Configuração OpenGL / GLFW

In [2]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
altura = 1000
largura = 1400
window = glfw.create_window(largura, altura, "Trabalho 1", None, None)
glfw.make_context_current(window)

In [3]:
vertex_code = """
        attribute vec3 position;
        attribute vec2 texture_coord;
        attribute vec3 normals;

        varying vec2 out_texture;
        varying vec3 out_fragPos;
        varying vec3 out_normal;
                
        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 projection;        
        
        void main(){
            gl_Position = projection * view * model * vec4(position,1.0);
            out_texture = vec2(texture_coord);
            out_fragPos = vec3(  model * vec4(position, 1.0));
            out_normal = vec3( transpose(inverse(model)) *vec4(normals, 1.0));    
        }
        """

fragment_code = """

        // parametro com a cor da(s) fonte(s) de iluminacao
        uniform vec3 lightPos; // define coordenadas de posicao da luz
        vec3 lightColor = vec3(1.0, 1.0, 1.0);
        
        // parametros da iluminacao ambiente e difusa
        uniform float ka; // coeficiente de reflexao ambiente
        uniform float kd; // coeficiente de reflexao difusa
        
        // parametros da iluminacao especular
        uniform vec3 viewPos; // define coordenadas com a posicao da camera/observador
        uniform float ks; // coeficiente de reflexao especular
        uniform float ns; // expoente de reflexao especular
        


        // parametros recebidos do vertex shader
        varying vec2 out_texture; // recebido do vertex shader
        varying vec3 out_normal; // recebido do vertex shader
        varying vec3 out_fragPos; // recebido do vertex shader
        uniform sampler2D samplerTexture;
        
        
        
        void main(){
        
            // calculando reflexao ambiente
            vec3 ambient = ka * lightColor;             
        
            // calculando reflexao difusa
            vec3 norm = normalize(out_normal); // normaliza vetores perpendiculares
            vec3 lightDir = normalize(lightPos - out_fragPos); // direcao da luz
            float diff = max(dot(norm, lightDir), 0.0); // verifica limite angular (entre 0 e 90)
            vec3 diffuse = kd * diff * lightColor; // iluminacao difusa
            
            // calculando reflexao especular
            vec3 viewDir = normalize(viewPos - out_fragPos); // direcao do observador/camera
            vec3 reflectDir = normalize(reflect(-lightDir, norm)); // direcao da reflexao
            float spec = pow(max(dot(viewDir, reflectDir), 0.0), ns);
            
            vec3 specular = ks * spec * lightColor;             
            
            // aplicando o modelo de iluminacao
            vec4 texture = texture2D(samplerTexture, out_texture);
            vec4 result = vec4((ambient + diffuse + specular),1.0) * texture; // aplica iluminacao
            gl_FragColor = result;

        }
        """

In [4]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

In [5]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)

In [6]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

## Funções auxiliares

In [7]:
def load_model_from_file(filename):
    """Loads a Wavefront OBJ file. """
    objects = {}
    vertices = []
    normals = []
    texture_coords = []
    faces = []

    material = None

    # abre o arquivo obj para leitura
    for line in open(filename, "r"): ## para cada linha do arquivo .obj
        if line.startswith('#'): continue ## ignora comentarios
        values = line.split() # quebra a linha por espaço
        if not values: continue


        ### recuperando vertices
        if values[0] == 'v':
            vertices.append(values[1:4])

        ### recuperando vertices
        if values[0] == 'vn':
            normals.append(values[1:4])

        ### recuperando coordenadas de textura
        elif values[0] == 'vt':
            texture_coords.append(values[1:3])

        ### recuperando faces 
        elif values[0] in ('usemtl', 'usemat'):
            material = values[1]
        elif values[0] == 'f':
            face = []
            face_texture = []
            face_normals = []
            for v in values[1:]:
                w = v.split('/')
                face.append(int(w[0]))
                face_normals.append(int(w[2]))
                if len(w) >= 2 and len(w[1]) > 0:
                    face_texture.append(int(w[1]))
                else:
                    face_texture.append(0)

            faces.append((face, face_texture, face_normals, material))

    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['faces'] = faces
    model['normals'] = normals

    return model



In [8]:
glHint(GL_LINE_SMOOTH_HINT, GL_DONT_CARE)
glEnable( GL_BLEND )
glBlendFunc( GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA )
glEnable(GL_LINE_SMOOTH)
glEnable(GL_TEXTURE_2D)
qtd_texturas = len(models_repo)
textures = glGenTextures(qtd_texturas)

def load_texture_from_file(texture_id, img_textura):
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)

    img = Image.open(img_textura)
    img_width = img.size[0]
    img_height = img.size[1]
    image_data = img.convert("RGBA").tobytes("raw", "RGBA",0,-1)

    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, img_width, img_height, 0, GL_RGBA, GL_UNSIGNED_BYTE, image_data)

## Carregando modelos

In [9]:
vertices_list = []    
normals_list = []
textures_coord_list = []

In [10]:
def process_obj(obj_filename, texture_filename, texture_id):
    modelo = load_model_from_file(f'modelos/{obj_filename}')

    initial_vertex = len(vertices_list)

    for face in modelo['faces']:
        for vertice_id in face[0]:
            vertices_list.append( modelo['vertices'][vertice_id-1] )
        
        for vertex_texture_id in face[1]:
            textures_coord_list.append( modelo['texture'][vertex_texture_id-1] )

        for normal_id in face[2]:
            normals_list.append( modelo['normals'][normal_id-1] )
    
    count_vertices = len(vertices_list) - initial_vertex

    load_texture_from_file(texture_id, f'modelos/{texture_filename}')

    return initial_vertex, count_vertices

In [11]:
for index, item in enumerate(models_repo.values()):
    item['texture_id'] = index
    init, count = process_obj(item['obj_filename'], item['texture_filename'], item['texture_id'])
    item['initial_vertex'] = init
    item['count_vertices'] = count

## Enviar dados para GPU

In [12]:
buffer = glGenBuffers(3)

In [13]:
vertices = np.zeros(len(vertices_list), [("position", np.float32, 3)])
vertices['position'] = vertices_list


glBindBuffer(GL_ARRAY_BUFFER, buffer[0])
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)

In [14]:
textures = np.zeros(len(textures_coord_list), [("position", np.float32, 2)]) # duas coordenadas
textures['position'] = textures_coord_list


glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)

In [15]:
normals = np.zeros(len(normals_list), [("position", np.float32, 3)]) # três coordenadas
normals['position'] = normals_list


# Upload coordenadas normals de cada vertice
glBindBuffer(GL_ARRAY_BUFFER, buffer[2])
glBufferData(GL_ARRAY_BUFFER, normals.nbytes, normals, GL_STATIC_DRAW)
stride = normals.strides[0]
offset = ctypes.c_void_p(0)
loc_normals_coord = glGetAttribLocation(program, "normals")
glEnableVertexAttribArray(loc_normals_coord)
glVertexAttribPointer(loc_normals_coord, 3, GL_FLOAT, False, stride, offset)

## Matrizes model, view e projection

In [16]:
def make_model_matrix(a_x, a_y, a_z, t_x, t_y, t_z, s_x, s_y, s_z):
    
    angleX = math.radians(a_x)
    angleY = math.radians(a_y)
    angleZ = math.radians(a_z)
    
    matrix_transform = glm.mat4(1.0) # instanciando uma matriz identidade
    
    # Escala
    matrix_transform = glm.translate(matrix_transform, glm.vec3(t_x, t_y, t_z))  
      
    # Rotação (deve ser na ordem Z, Y e X)
    matrix_transform = glm.rotate(matrix_transform, angleZ, glm.vec3(0, 0, 1))
    matrix_transform = glm.rotate(matrix_transform, angleY, glm.vec3(0, 1, 0))
    matrix_transform = glm.rotate(matrix_transform, angleX, glm.vec3(1, 0, 0))
    
    # Translação
    matrix_transform = glm.scale(matrix_transform, glm.vec3(s_x, s_y, s_z))
    
    return np.array(matrix_transform)

def make_view_matrix():
    global cameraPos, cameraFront, cameraUp
    mat_view = glm.lookAt(cameraPos, cameraPos + cameraFront, cameraUp);
    return np.array(mat_view)

def make_projection_matrix():
    global altura, largura, inc_fov, inc_near, inc_far
    mat_projection = glm.perspective(glm.radians(45.0), largura/altura, 0.1, 1000.0)
    return np.array(mat_projection)

## Funções de desenho

In [17]:
def draw_model(model_data):
    initial_vertex, count_vertices, texture_id, translation, angles, scale, draw_type = itemgetter(
        'initial_vertex', 
        'count_vertices', 
        'texture_id', 
        'translation', 
        'angles',
        'scale',
        'draw_type'
    )(model_data)
    
    model_matrix = make_model_matrix(angles[0], angles[1], angles[2],
                                translation[0], translation[1], translation[2],
                                scale[0], scale[1], scale[2])

    model_location = glGetUniformLocation(program, "model")
    
    glUniformMatrix4fv(model_location, 1, GL_TRUE, model_matrix)
    glBindTexture(GL_TEXTURE_2D, texture_id)

    if draw_type == 'TRIANGLE':
        glDrawArrays(GL_TRIANGLES, initial_vertex, count_vertices)
    else:
        glDrawArrays(GL_QUADS, initial_vertex, count_vertices)

## Funções de atualização

In [18]:
# Simple rotation and oscillating scale animation
def update_big_skull(model_data):
    # Update the angles (rotation)
    model_data['angles'][1] += 0.5
    
    # == Scale animation ==
    
    # Save the original scale
    if not ('original_scale' in model_data):
        model_data['original_scale'] = model_data['scale'][:]
    
    # Initialize the param_t
    if 'param_t' not in model_data:
        model_data['param_t'] = 0.0
    
    # Update the param_t
    model_data['param_t'] += 0.1
    model_data['param_t'] = model_data['param_t'] % (2 * math.pi)
    
    # Calculate scale factors
    scale_factor = 0.06 * math.sin(model_data['param_t'])
    
    #Update the scale
    model_data['scale'] = [
        model_data['original_scale'][0] + scale_factor,
        model_data['original_scale'][1] + scale_factor,
        model_data['original_scale'][2] + scale_factor
    ]
    
    
# Simple oscillating translation and rotation animation
def update_cat(model_data):
    # Save the original translation
    if 'original_translation' not in model_data:
        model_data['original_translation'] = model_data['translation'][:]
        
    # Save the original angles
    if 'original_angles' not in model_data:
        model_data['original_angles'] = model_data['angles'][:]
    
    # Initialize the param_t
    if 'param_t' not in model_data:
        model_data['param_t'] = 0.0
    
    # Update the param_t
    model_data['param_t'] += 0.06
    model_data['param_t'] = model_data['param_t'] % (2 * math.pi)
    
    
    # Calculate offsets
    offsetX = 0.1 * math.cos(model_data['param_t'])
    offsetY = 0.1 * math.sin(model_data['param_t'])
    offsetZ = 0.1 * math.sin(model_data['param_t'])
    
    rotation = (1 + math.sin(model_data['param_t'])) * 10
        
    # Update the model
    model_data['translation'] = [
        model_data['original_translation'][0] + offsetX,
        model_data['original_translation'][1] + offsetZ,
        model_data['original_translation'][2] + offsetY,
    ]
    model_data['angles'] = [
        model_data['original_angles'][0],
        model_data['original_angles'][1],
        model_data['original_angles'][2] + rotation
    ]
    
# Always rotates to face the camera
def update_small_skull(model_data):
    if 'original_angles' not in model_data:
        model_data['original_angles'] = model_data['angles'][:]
    
    # Axis Y rotation
    y_rotation = math.degrees(
        math.atan2(
            cameraPos[0] - model_data['translation'][0],
            cameraPos[2] - model_data['translation'][2]
        )
    ) - 45
    model_data['angles'][1] = y_rotation + model_data['original_angles'][1]
    
    # Axis X rotation
    x_rotation = math.degrees(
        math.atan2(
            cameraPos[1] - model_data['translation'][1],
            cameraPos[2] - model_data['translation'][2]
        )
    ) - 15
    model_data['angles'][0] = model_data['original_angles'][0] - x_rotation
    
    
update_functions = {
    'big_skull': update_big_skull,
    'cat': update_cat,
    'small_skull': update_small_skull,
}
    
def update_model(model_key, model_data):
    if model_key in update_functions:
        update_functions[model_key](model_data)

## Desenhar luz

In [19]:
def desenha_luz(t_x, t_y, t_z):
    

    
    r_x = 0.0; r_y = 0.0; r_z = 1.0;
    
    # translacao
    t_x = 0.0; t_y = 0.0; t_z = 0.0;
    
    # escala
    s_x = 0.1; s_y = 0.1; s_z = 0.1;
    
    mat_model = make_model_matrix(r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    
    #### define parametros de ilumincao do modelo
    ka = 1 # coeficiente de reflexao ambiente do modelo
    kd = 1 # coeficiente de reflexao difusa do modelo
    ks = 1 # coeficiente de reflexao especular do modelo
    ns = 1000.0 # expoente de reflexao especular
    
    loc_ka = glGetUniformLocation(program, "ka") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_ka, ka) ### envia ka pra gpu
    
    loc_kd = glGetUniformLocation(program, "kd") # recuperando localizacao da variavel kd na GPU
    glUniform1f(loc_kd, kd) ### envia kd pra gpu    
    
    loc_ks = glGetUniformLocation(program, "ks") # recuperando localizacao da variavel ks na GPU
    glUniform1f(loc_ks, ks) ### envia ns pra gpu        
    
    loc_ns = glGetUniformLocation(program, "ns") # recuperando localizacao da variavel ns na GPU
    glUniform1f(loc_ns, ns) ### envia ns pra gpu            
    
    loc_light_pos = glGetUniformLocation(program, "lightPos") # recuperando localizacao da variavel lightPos na GPU
    glUniform3f(loc_light_pos, t_x, t_y, t_z) ### posicao da fonte de luz
        
    
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 1)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 36, 36) ## renderizando
    


## Definir eventos

In [20]:
cameraPos = glm.vec3(0.0,  5.0,  1.0)
cameraFront = glm.vec3(0.0,  -1.0, -1.0)
cameraUp = glm.vec3(0.0,  1.0,  0.0)

cameraSpeed = 0.2

isPolygonalMode = False

def key_event(window, key, scancode, action, mods):
    global cameraPos, cameraFront, cameraUp, isPolygonalMode
        
        
    if key == 87 and (action==1 or action==2): # tecla W
        cameraPos += cameraSpeed * cameraFront
    
    if key == 83 and (action==1 or action==2): # tecla S
        cameraPos -= cameraSpeed * cameraFront
    
    if key == 65 and (action==1 or action==2): # tecla A
        cameraPos -= glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if key == 68 and (action==1 or action==2): # tecla D
        cameraPos += glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed

    if key == 80 and action == 1:
        isPolygonalMode = not isPolygonalMode
        
    # Enforce camera bounds
    cameraPos[0] = max(-190, min(190, cameraPos[0]))
    cameraPos[1] = max(0.5, min(60, cameraPos[1]))
    cameraPos[2] = max(-190, min(190, cameraPos[2]))
        
        
        
firstMouse = True
yaw = -90.0 
pitch = 0.0
lastX =  largura/2
lastY =  altura/2

def mouse_event(window, xpos, ypos):
    global firstMouse, cameraFront, yaw, pitch, lastX, lastY
    if firstMouse:
        lastX = xpos
        lastY = ypos
        firstMouse = False

    xoffset = xpos - lastX
    yoffset = lastY - ypos
    lastX = xpos
    lastY = ypos

    sensitivity = 0.3 
    xoffset *= sensitivity
    yoffset *= sensitivity

    yaw += xoffset;
    pitch += yoffset;

    
    if pitch >= 90.0: pitch = 90.0
    if pitch <= -90.0: pitch = -90.0

    front = glm.vec3()
    front.x = math.cos(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    front.y = math.sin(glm.radians(pitch))
    front.z = math.sin(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    cameraFront = glm.normalize(front)


    
glfw.set_key_callback(window,key_event)
glfw.set_cursor_pos_callback(window, mouse_event)


## Lógica principal

In [21]:
glfw.show_window(window)
glfw.set_cursor_pos(window, lastX, lastY)

In [22]:
glEnable(GL_DEPTH_TEST) 

ang = 0.1
ns_inc = 32

while not glfw.window_should_close(window):

    glfw.poll_events() 
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glClearColor(1.0, 1.0, 1.0, 1.0)
    glPolygonMode(GL_FRONT_AND_BACK, GL_LINE if isPolygonalMode else GL_FILL)
    
    for model_name, model_data in models_repo.items():
        update_model(model_name, model_data)
        draw_model(model_data)

    ang += 0.0005
    desenha_luz(math.cos(ang)*0.5, math.sin(ang)*0.5, 3.0)
        
    mat_view = make_view_matrix()
    loc_view = glGetUniformLocation(program, "view")
    glUniformMatrix4fv(loc_view, 1, GL_TRUE, mat_view)

    mat_projection = make_projection_matrix()
    loc_projection = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_projection, 1, GL_TRUE, mat_projection)    
    
    glfw.swap_buffers(window)

glfw.terminate()